In [ ]:
import pandas as pd
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
list_cluster= glob.glob('/content/drive/MyDrive/Colab Notebooks/km25/*clean*')

In [ ]:
print(list_cluster)

['/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_3_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_1_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_9_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_19_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_2_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_14_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_20_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_11_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_6_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_17_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_18_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_16_clean_msg.txt', '/content/drive/MyDrive/Colab Notebooks/km25/km_cluster_21_clean_msg.txt', '/content/drive/MyDrive/Colab

In [ ]:
def read_cluster(files):
  dic={}
  for i,cluster in enumerate(files):
    with open(cluster, "r") as file:
      dic[i] = file.readlines()
  return dic

In [ ]:
dic= read_cluster(list_cluster)
print(len(dic))

25


In [ ]:
print(len(dic[24]))

60288


In [ ]:
for i in range(len(list_cluster)):
  vars()["data_"+str(i)]=pd.DataFrame(dic[i], columns = ['tweet'])

In [ ]:
print(data_3.head(15))

                                                tweet
0                                          patience\n
1                                      coeur survit\n
2                                             coeur\n
3                    aime elis pouvoir décrir amour\n
4                                           dignité\n
5                                    soutien pleure\n
6      créativité sincérité humour amour incroyable\n
7    demander indemnité séquelle vie mourir bofooxi\n
8             aime cité aller méchantsx amour haine\n
9                                              mort\n
10                              abonné wahou gloire\n
11  meilleur inutile bénéfique pacifier entendre b...
12                                             mère\n
13  meilleur verset inutile ligne bénéfique pacifi...
14  réciterait stance inutile valoir mieux réciter...


In [ ]:
!pip install gensim==4.0

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
w2v = KeyedVectors.load('/content/drive/MyDrive/Colab Notebooks/w2vec_model_d300_1M')

In [ ]:
import numpy as np

In [ ]:
#encode un message
def encode(msg):
    return np.mean([w2v[word] for word in msg if word in w2v] or [np.zeros(300)], axis = 0)

In [ ]:
w2v.similar_by_word("sale")

[('sal', 0.6782066822052002),
 ('pvte', 0.629138708114624),
 ('mèrde', 0.6162537336349487),
 ('pute', 0.6096962094306946),
 ('pte', 0.6044666767120361),
 ('clochard', 0.6014346480369568),
 ('slp', 0.6004031896591187),
 ('crasseux', 0.5957321524620056),
 ('puent', 0.5908592939376831),
 ('daronn', 0.5900101661682129)]

In [ ]:
def encoded(dic):
  w2v={}
  for i in range(len(dic)):
    w2v[i]=np.array([encode(msg) for msg in dic[i]])
  return w2v

In [ ]:
%%time
#encodage de l'échantillon pour créer le jeu de données
dataset_encoded = encoded(dic)

CPU times: user 44.9 s, sys: 1.21 s, total: 46.1 s
Wall time: 46.1 s


In [ ]:
print(len(dataset_encoded[24]))

60288


In [ ]:
#sauvegarder mes embeddings
%%time
"""for i in range(len(dataset_encoded)):
  with open(f"w2v/w2v{i}.txt", 'a') as f:
      for row in dataset_encoded[i]:
        #np.savetxt(f, row)"""

In [ ]:
from sklearn.ensemble import IsolationForest

clf=IsolationForest(n_estimators=100, max_samples='auto', \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)


In [ ]:
%%time
for i in range(len(dataset_encoded)):
  clf.fit(dataset_encoded[i])

CPU times: user 4min 38s, sys: 3.86 s, total: 4min 42s
Wall time: 2min 27s


In [ ]:
%%time
pred={}
for i in range(len(dataset_encoded)):
  pred[i] = clf.predict(dataset_encoded[i])

CPU times: user 1min 47s, sys: 4.66 s, total: 1min 52s
Wall time: 1min 52s


In [ ]:
print(pred[1])

[1 1 1 ... 1 1 1]


In [ ]:
for i in range(len(dataset_encoded)):
  vars()["data_"+str(i)]['pred']= pred[i]

In [ ]:
print(data_1.head(20))

                                                tweet  pred
0   ethusd dly doublement cadrer ich ratio ext fib...     1
1               policier convaincre lfi côté plaque\n     1
2                                       heure crime\n     1
3   mars baye cheikh diop âgé an tuer balle garage...    -1
4   mars mansour thiam tuer balle colobane luire v...     1
5                                  fautes ca france\n     1
6   vidéo élection voler dominion faveur macron pr...     1
7   autorité moral financement étranger national a...    -1
8                                         éducation\n     1
9                                          allemand\n     1
10                                   final européen\n     1
11                   lien corruption politique mdrr\n     1
12  minute temps accorder écologie débat primaire ...     1
13  respect parti politique geopo droit bien viser...     1
14  euro primeinflation pauvre décider sénateur pr...     1
15                   euro prime noël sén

In [ ]:
outliers={}
for i in range(len(dataset_encoded)):
  outliers[i]=vars()["data_"+str(i)].loc[vars()["data_"+str(i)]['pred']==-1]
  outlier_index=list(outliers[i].index)
#print(outlier_index)
#Find the number of anomalies and normal points here points classified -1 are anomalous
  print("cluster"+str(i)+":"+"\n")
  print(vars()["data_"+str(i)]['pred'].value_counts())
  print("\n")

cluster0:

 1    10083
-1     1079
Name: pred, dtype: int64


cluster1:

 1    36297
-1    13362
Name: pred, dtype: int64


cluster2:

 1    10628
-1      327
Name: pred, dtype: int64


cluster3:

 1    17147
-1     4296
Name: pred, dtype: int64


cluster4:

 1    26157
-1      527
Name: pred, dtype: int64


cluster5:

 1    2829
-1     136
Name: pred, dtype: int64


cluster6:

 1    969
-1     15
Name: pred, dtype: int64


cluster7:

 1    1637
-1      33
Name: pred, dtype: int64


cluster8:

 1    12099
-1      861
Name: pred, dtype: int64


cluster9:

 1    2043
-1      58
Name: pred, dtype: int64


cluster10:

 1    15449
-1     4698
Name: pred, dtype: int64


cluster11:

 1    3131
-1      56
Name: pred, dtype: int64


cluster12:

 1    7052
-1     964
Name: pred, dtype: int64


cluster13:

 1    3095
-1     127
Name: pred, dtype: int64


cluster14:

 1    3701
-1      58
Name: pred, dtype: int64


cluster15:

 1    31439
-1     1847
Name: pred, dtype: int64


cluster16:

 1    16

In [ ]:
for i in range(len(outliers)):
  with open(f"outliers{i}", 'a') as f:
    f.write(str(outliers[i]))

In [ ]:
for i in range(len(outliers)):
  with open(f"drive/MyDrive/outliers25/outliers{i}", 'a') as f:
    dfAsString = outliers[i].to_string(header=False, index=False)
    f.write(dfAsString)